In [ ]:
import numpy as np
import matplotlib.pyplot as plt

In [ ]:
# Define trigonometric function to integrate
def fx_sin(x):
    import math
    # print("fx evlauted at x=",x)
    return math.sin(x)

# Lets define its analytic integral is we know it
def a_int_fx_sin(x):
    import math
    return -math.cos(x)

In [ ]:
# Define more interesting function to integrate
def fx_pow(x):
    # print("fx evlauted at x=",x)
    return 4.*x**3

# Lets define its analytic integral is we know it
def a_int_fx_pow(x):
    return x**4

In [ ]:
# Define simple function to integrate
def fx_lin(x):
        # print("fx evlauted at x=",x)
        return 2*x
    
# Lets define its analytic integral is we know it
def a_int_fx_lin(x):
        return x**2

In [ ]:
# Select a function
func_to_use="trig"
# func_to_use="pow"
# func_to_use="lin"

if func_to_use == "lin":
    def fx(x):
        return fx_lin(x)
    def a_int_fx(x):
        return a_int_fx_lin(x)
    
if func_to_use == "pow":
    def fx(x):
        return fx_pow(x)
    def a_int_fx(x):
        return a_int_fx_pow(x)    
    
if func_to_use == "trig":
    def fx(x):
        return fx_sin(x)
    def a_int_fx(x):
        return a_int_fx_sin(x)    

In [ ]:
# Create a sequence of discrete values for evaluating function.
xvals= np.linspace(0,5,25)
dx=xvals[1]-xvals[0]

# Plot the function and its integral at the sample points (we know the closed form integral in these examples)
fig=plt.figure(figsize=(16, 12))
ax = [fig.add_subplot(1,2,i+1) for i in range(2)]
# Function
ax[0].plot(xvals,[fx(x) for x in xvals],'o-',markersize=12);
ax[0].set_xlabel("x")
ax[0].set_ylabel("f(x)");
# Integral
ax[1].plot(xvals,[a_int_fx(x) for x in xvals],'o-',markersize=12);
ax[1].set_xlabel("x")
ax[1].set_ylabel("int_f(x)");

In [ ]:
# Routines to help plot the integration approximating rectangles.
def vline(x,fx,dx):
    xlo,ylo=x,0
    xhi,yhi=x,fx(x)
    xmidhi,ymidhi=x,0.5*(fx(x)+fx(x+dx))
    xmidlo,ymidlo=x,0.5*(fx(x)+fx(x-dx))
    return [xlo, xhi, xmidhi, xmidlo], [ylo, yhi, ymidhi, ymidlo]

def hline(x,fx,dx):
    ymid=0.5*(fx(x)+fx(x+dx))
    xlo,ylo=x,ymid
    xhi,yhi=x+dx,ymid
    return [xlo,xhi],[ylo,yhi]

In [ ]:
# Make a illustrative plot of function and simple integration approximation rectangles
plt.figure(figsize=(16, 12))

# Plot function evlauted at the xval values
plt.plot(xvals,[fx(x) for x in xvals],'o',markersize=12)

# Not draw vertical and horizonal lines for rectangles
xval=xvals[0]
vl=vline(xval,fx,dx)
plt.plot([xval,xval],[vl[1][i] for i in [0,2]],'k')
for xval in xvals[1:-1]:
    vl=vline(xval,fx,dx);plt.plot(vl[0][0:4],[vl[1][i] for i in [0,1,2,3]],'k')
xval=xvals[-1]
plt.plot([xval,xval],[vl[1][i] for i in [0,2]],'k')
for xval in xvals[:-1]:   
    hl=hline(xval,fx,dx);plt.plot(hl[0],hl[1],'k')
plt.xlabel("x")
plt.ylabel("f(x)");

In [ ]:
# Now lets make a very simple integrator function
def myint(fx, xs ):
    total_a=0
    dx=xs[1]-xs[0]
    for xval in xs[:-1]:
        midy=0.5*(fx(xval)+fx(xval+dx))
        a=midy*dx
        total_a=total_a+a
    return total_a

In [ ]:
# Lets see how well it works
ni=myint(fx,xvals)
ai=( a_int_fx(xvals[-1]) - a_int_fx(xvals[0]) )
print("Numerical integral =", ni  )
print("Error = ", ni - ai )
print("Percent error =",(ni-ai)/(0.5*(ni+ai))*100,"%")

In [ ]:
# The Scipy package provide a built-in numerical integrator that fits higher order curves to the function
# evaluation to come up with a better fit for an integral.
#
# The actual usage is quite similar i.e. define a function to evaluate and pass in to "integrator"
#
# The integrator is written for a general function dy/dt=f(y,t) so we create a stub function
# to use our simple fx(x) defined earlier.
#
from scipy.integrate import odeint
def fgenx(y,t):
    return fx(t)
ys = odeint(fgenx, 0, xvals )
ni=ys[-1]
print("Numerical integral =", ni  )
print("Error = ", ni - ai )
print("Percent error =",(ni-ai)/(0.5*(ni+ai))*100,"%")
errvec=np.abs(ys.flatten()-[ a_int_fx(xv) for xv in xvals]) + a_int_fx(xvals[0])
plt.plot(errvec);

In [ ]:
# The scipy odeintegrator also allows us to specify a desired accuracy to try and achieve
# For different accuracies it will use different step sizes and algorithms.
ys = odeint(fgenx, 0, xvals, atol=1.e-16)
ni=ys[-1]
print("Numerical integral =", ni  )
print("Error = ", ni - ai )
print("Percent error =",(ni-ai)/(0.5*(ni+ai))*100,"%")
errvec=ys.flatten()-[ a_int_fx(xv) for xv in xvals] + a_int_fx(xvals[0])
plt.plot(errvec);

### Internally odeint uses code from a library called lsoda ( https://github.com/cran/odesolve/blob/master/src/lsoda.f ). The inner workings of this library are quite complex and are somewhat doumented in the articles by Alan Hindmarsh ( https://computing.llnl.gov/sites/default/files/ODEPACK_pub1_u88007.pdf )  and Linda Petzold ( https://cse.cs.ucsb.edu/sites/cse.cs.ucsb.edu/files/publications/SCE000136.pdf ).